In [2]:
import pandas as pd

In [3]:
Alias=pd.read_csv('data/Alias.csv')
Alias

,VC_fullname,Alias
0,10T Holdings,10T
1,123RF LIMITED,123RF
2,1535创投,杭州壹伍叁伍创业投资管理有限公司
3,1982 Ventures Pte.Ltd.,1982 Ventures
4,20/20 HealthCare Partners,20/20 HealthCare
...,...,...
43423,龙芯中科技术股份有限公司,龙芯中科技术
43424,龙芯中科技术股份有限公司,龙芯中科技术有限公司
43425,龙赢富泽资产管理(北京)有限公司,龙赢富泽资产
43426,龙赢富泽资产管理(北京)有限公司,龙赢富泽资产管理（北京）有限公司


In [4]:
ChinaVCtype=pd.read_csv('data/ChinaVCtype.csv')
ChinaVCtype

,VC_fullname,Unified_social_credit_ID,GVCagency_ne,GVCagency_equity,GVCsoe_ne,GVCsoe_equity,IVC_ne,CVC_ne,BVC_ne,FVC_ne,...,UVC_excl,PenVC_excl,Unknown,Direct_judge,Corporate_parent,Parent_unified_social_credit_ID,Parent_industry_qcc,Parent_listed,Parent_listedmarket,Parent_stockcode
0,1535创投,91330102341958567N,0.0,0.0,0.0,0.000000,1.0,1.0,0.0,0.0,...,0,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN
1,3M New Ventures,NaN,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,...,0,0,0,No,明尼苏达矿业制造（中国）投资有限公司,913100006073666908,租赁和商务服务业,NaN,NaN,NaN
2,"Amino Capital Management Company,LLC",NaN,0.0,0.0,1.0,99.989998,1.0,0.0,0.0,1.0,...,0,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN
3,AppWorks Ventures之初创投,NaN,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,...,0,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN
4,Ares Management LLC,NaN,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,...,0,0,0,No,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49182,龙旗控股,91460000MA5U1HC28G,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,Human Coding,NaN,NaN,NaN,NaN,NaN,NaN
49183,龙洲股份,91350000753125975Q,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,Human Coding,NaN,NaN,NaN,NaN,NaN,NaN
49184,龙马新三板进取基金1号（管理）,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,Human Coding,NaN,NaN,NaN,NaN,NaN,NaN
49185,龚洁伟,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,Human Coding,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
ChinaVCtype.columns

Index(['VC_fullname', 'Unified_social_credit_ID', 'GVCagency_ne',
       'GVCagency_equity', 'GVCsoe_ne', 'GVCsoe_equity', 'IVC_ne', 'CVC_ne',
       'BVC_ne', 'FVC_ne', 'UVC_ne', 'PenVC_ne', 'GVC_excl', 'GVC_equity',
       'CVC_excl', 'IVC_excl', 'BVC_excl', 'FVC_excl', 'UVC_excl',
       'PenVC_excl', 'Unknown', 'Direct_judge', 'Corporate_parent',
       'Parent_unified_social_credit_ID', 'Parent_industry_qcc',
       'Parent_listed', 'Parent_listedmarket', 'Parent_stockcode'],
      dtype='object')

现在这部分数据里面有两个文件，一个是Alias，一个是ChinaVCtype，两个文件是通过vcfullname连接的，现在要先把这两个文件整合，整合完之后和invest_all的文件里面的拆分后投资方名称匹配（on=alias），匹配到的字段包括'VC_fullname','GVCagency_ne','GVCsoe_ne','GVC_excl'。也就是现在根据投资方名称先把投资事件拆分成不同投资方，然后再根据不同的投资方的名称和GVC type里面的alias匹配，新加入'VC_fullname','GVCagency_ne','GVCsoe_ne','GVC_excl'这几个字段。之后再把他们整合到投资事件层面，整合的规则是：1）GVC_DUMMY: 对于每一个投资事件，只要有一个投资方里面的GVC_excl是1，则这个投资事件的GVC就是1，2）GVC_Continuous：对于每一个投资事件，计算投资方中GVC_excl是1的比例

In [15]:
ChinaVCtype.VC_fullname.nunique()

49187

In [8]:
Alias.groupby('Alias').VC_fullname.nunique().sort_values()

Alias
(Ad)Ventures    1
108度集团          1
10T             1
11.2 Capital    1
123RF           1
               ..
通和资本            2
源渡创投            2
微光创投            2
日本SBI控股株式会社     2
溪山天使会           2
Name: VC_fullname, Length: 43305, dtype: int64

In [9]:
Alias[Alias.Alias=='溪山天使会']

,VC_fullname,Alias
36418,溪山股权投资基金管理(深圳)有限公司,溪山天使会
36419,溪山股权投资基金管理（深圳）有限公司,溪山天使会


In [10]:
Alias[Alias.Alias=='日本SBI控股株式会社']

,VC_fullname,Alias
1785,SBI Holdings,日本SBI控股株式会社
26403,日本SBI控股株式会社北京代表处,日本SBI控股株式会社


In [16]:
Alias.groupby('VC_fullname').Alias.nunique().sort_values()

VC_fullname
4BIO Partners LLP            1
123RF LIMITED                1
1535创投                       1
1982 Ventures Pte.Ltd.       1
20/20 HealthCare Partners    1
                            ..
安吉兴平股权投资合伙企业（有限合伙）           3
广东德和信企业科技有限公司                3
简道众创投资股份有限公司                 3
宁波创达创业投资合伙企业（有限合伙）           3
贵州省科技风险投资有限公司                3
Name: Alias, Length: 41283, dtype: int64

In [17]:
Alias[Alias.VC_fullname=='贵州省科技风险投资有限公司']

,VC_fullname,Alias
40721,贵州省科技风险投资有限公司,贵州省科技风险投资公司
40722,贵州省科技风险投资有限公司,贵州风投
40723,贵州省科技风险投资有限公司,贵阳市科技风险投资有限公司


In [18]:
Alias[Alias.VC_fullname=='宁波创达创业投资合伙企业（有限合伙）']

,VC_fullname,Alias
20274,宁波创达创业投资合伙企业（有限合伙）,创达创业投资
20275,宁波创达创业投资合伙企业（有限合伙）,苏州创达创业投资企业(有限合伙)
20276,宁波创达创业投资合伙企业（有限合伙）,衢州创达投资合伙企业（有限合伙）


In [22]:
Alias[Alias.Alias=='创达创业投资']

,VC_fullname,Alias
20274,宁波创达创业投资合伙企业（有限合伙）,创达创业投资


In [7]:
map_fullname = Alias.groupby('VC_fullname')['Alias'].unique().map(';'.join)
map_fullname

VC_fullname
10T Holdings                                     10T
123RF LIMITED                                  123RF
1535创投                              杭州壹伍叁伍创业投资管理有限公司
1982 Ventures Pte.Ltd.                 1982 Ventures
20/20 HealthCare Partners           20/20 HealthCare
                                      ...           
龙腾荣盛投资管理有限公司                                  龙腾荣盛投资
龙腾资本有限公司                                        龙腾资本
龙芯中科技术股份有限公司                       龙芯中科技术;龙芯中科技术有限公司
龙赢富泽资产管理(北京)有限公司             龙赢富泽资产;龙赢富泽资产管理（北京）有限公司
龙金（北京）投资管理有限公司                            龙金（北京）投资管理
Name: Alias, Length: 41283, dtype: object

In [9]:
map_fullname.loc['宁波创达创业投资合伙企业（有限合伙）']

'创达创业投资;苏州创达创业投资企业(有限合伙);衢州创达投资合伙企业（有限合伙）'

In [11]:
ChinaVCtype['Alias']=ChinaVCtype['VC_fullname'].map(map_fullname)
ChinaVCtype['Alias'].count()

np.int64(40754)

In [14]:
ChinaVCtype.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49187 entries, 0 to 49186
Data columns (total 29 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   VC_fullname                      49187 non-null  object 
 1   Unified_social_credit_ID         31871 non-null  object 
 2   GVCagency_ne                     43681 non-null  float64
 3   GVCagency_equity                 34731 non-null  float64
 4   GVCsoe_ne                        43681 non-null  float64
 5   GVCsoe_equity                    34731 non-null  float64
 6   IVC_ne                           36458 non-null  float64
 7   CVC_ne                           36458 non-null  float64
 8   BVC_ne                           36458 non-null  float64
 9   FVC_ne                           36458 non-null  float64
 10  UVC_ne                           36458 non-null  float64
 11  PenVC_ne                         36458 non-null  float64
 12  GVC_excl          

In [21]:
ChinaVCtype['Parent_listed'].unique(),ChinaVCtype['Corporate_parent'].nunique()

(array([nan, 'Listed', 'Delisted'], dtype=object), 3146)

In [13]:
ChinaVCtype.to_csv('data/ChinaVCtype.csv')

In [14]:
invest_all=pd.read_excel('data/source/invest_all.xlsx',index_col=0)
invest_all.columns

Index(['企业简称', '企业全称', '业务描述', '所在行业', '所在地', '企业简介', '详细地址', '联系电话', '企业网址',
       '邮箱', '成立时间', '统一信用代码', '法人代表', '注册资本', '经营状态', '经营范围', '注册地址', '上市状态',
       '股票代码', '交易所', '交易轮次', '交易金额', '单位', '交易股权', '投后估值', '单位.1', '交易时间',
       '交易状态', '本次交易股份数量', '每股价格', '单位.2', '交易类型', '交易方式', '支付方式', '是否关联交易',
       '重大资产重组', '是否跨境交易', '投资方名称', '投资方基金名称', '投资机构类型', '交易地区类型', '投资金额',
       '单位.3', '本次交易股权比例', '投资方上市状态', '投资方股票代码', '投资方交易所', '退出方名称', '退出方基金名称',
       '退出机构类型', '退出方式', '退出方上市状态', '退出方股票代码', '退出方交易所', '中介名称', '中介类型',
       '投资方名称_A', '投资方基金名称_A', '政府引导基金的管理机构_机构全称', '政府引导基金的管理机构_机构简称', '行业',
       '子行业'],
      dtype='object')

In [24]:
invest_all['投资方名称']

0                          上海领汇创投
1         祥峰投资中国基金 ;火山石投资 ;西藏高榕资本
2                   金顺东投资 ;奥都百升网络
3                            真格基金
4                            时代伯乐
                   ...           
155304                   富华睿银基金管理
155305                        深创投
155306                   CVC Asia
155307                       四维图新
155308                      及时雨创投
Name: 投资方名称, Length: 155309, dtype: object

In [30]:
invest_all['投资方名称_A']=invest_all['投资方名称'].str.split(';').dropna().map(lambda x:[i.strip() for i in x])
invest_all['投资方名称_A']

0                          [上海领汇创投]
1         [祥峰投资中国基金, 火山石投资, 西藏高榕资本]
2                   [金顺东投资, 奥都百升网络]
3                            [真格基金]
4                            [时代伯乐]
                    ...            
155304                   [富华睿银基金管理]
155305                        [深创投]
155306                   [CVC Asia]
155307                       [四维图新]
155308                      [及时雨创投]
Name: 投资方名称_A, Length: 155309, dtype: object

In [40]:
map_alias_dict = Alias.groupby('Alias')['VC_fullname'].unique().to_dict()
alias_set = set(map_alias_dict.keys())

In [38]:
invest_all['ChinaVC_alias']=invest_all['投资方名称_A'].dropna().apply(lambda x:';'.join(set(x)&alias_set)).replace('',pd.NA)
invest_all['ChinaVC_alias'].str.split(';').explode().nunique(),invest_all['ChinaVC_alias'].isna().sum()

(36057, np.int64(36884))

In [75]:
invest_all['VC_fullname'] =invest_all['ChinaVC_alias'].dropna().map(lambda x: ';'.join({fullname for alias in x.split(';') for fullname in map_alias_dict[alias]}))
invest_all['VC_fullname'] 

0                          上海领汇创业投资有限公司
1                          西藏高榕资本管理有限公司
2          上海金顺东投资管理有限公司;北京奥都百升网络科技有限公司
3         北京真格天成投资管理有限公司;天津真格天地投资管理有限公司
4                     深圳市时代伯乐创业投资管理有限公司
                      ...              
155304                              NaN
155305                    深圳市创新投资集团有限公司
155306            CVC Asia Pacific,Ltd.
155307                   北京四维图新科技股份有限公司
155308                    深圳市汤乔电子科技有限公司
Name: VC_fullname, Length: 155309, dtype: object

In [49]:
vc2gvc = ChinaVCtype.set_index('VC_fullname')['GVC_excl'].to_dict()
invest_all['GVC'] = invest_all['VC_fullname'].dropna().str.split(';').map( lambda vc_list: [vc2gvc.get(vc, 0) for vc in vc_list])# if vc in vc2gvc
invest_all['GVC'] 

0            [0]
1            [0]
2         [0, 0]
3         [0, 0]
4            [0]
           ...  
155304       NaN
155305       [1]
155306       [0]
155307       [0]
155308       [0]
Name: GVC, Length: 155309, dtype: object

In [51]:
invest_all['GVC'].explode().sum()

28186

In [57]:
invest_all['GVC_DUMMY']=invest_all['GVC'].dropna().map(any).astype(int)#.map(lambda x: 1 if 1 in x else 0)
invest_all['GVC_DUMMY'].sum()

np.float64(22923.0)

In [62]:
import numpy as np
invest_all['GVC_Continuous']= invest_all['GVC'].dropna().map(np.mean)#.map(lambda x: sum(x) / len(x))
invest_all['GVC_Continuous'].mean()

np.float64(0.13929885744424478)

In [ ]:
invest_all['GVC_Continuous'].describe()

In [67]:
invest_all['GVC']=invest_all['GVC'].dropna().map(lambda x: ';'.join(str(i).strip() for i in x) )

In [69]:
invest_all.to_excel('data/invest_all_gvc.xlsx')

In [76]:
invest_all['VC_fullname']=invest_all['VC_fullname'].dropna().str.split(';')
invest_vc=invest_all.dropna(subset=['VC_fullname']).explode('VC_fullname')
invest_vc['VC_fullname'].isna().sum(),invest_vc.shape

(np.int64(0), (218426, 67))

In [88]:
(invest_vc['VC_fullname']=='').sum()

np.int64(0)

In [83]:
ChinaVCtype[['VC_fullname','GVCagency_ne','GVCsoe_ne','GVC_excl']].duplicated().sum()

np.int64(0)

In [85]:
invest_vc=invest_vc.merge(ChinaVCtype[['VC_fullname','GVCagency_ne','GVCsoe_ne','GVC_excl']],on='VC_fullname',how='left')
invest_vc.shape

(218426, 70)

In [90]:
invest_vc.to_csv('data/invest_all_vctype.csv')